In [ ]:
!pip install spacy gensim
from spacy.cli import download

download("en_core_web_sm")

/media/scientist-anand/volume/mr_document/all_venv/chatterbox_venv/bin/python: No module named spacy


In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
text = "Natural Language Processing helps computers understand human language."
doc = nlp(text)
print(doc)

Natural Language Processing helps computers understand human language.


In [10]:
clean_data = []
for token in doc:
    if token.is_alpha and not token.is_stop:
        print(token)
        print("lemma==>", token.lemma_.lower())
        clean_data.append(token.lemma_.lower())

print(clean_data)

Natural
lemma==> natural
Language
lemma==> language
Processing
lemma==> processing
helps
lemma==> help
computers
lemma==> computer
understand
lemma==> understand
human
lemma==> human
language
lemma==> language
['natural', 'language', 'processing', 'help', 'computer', 'understand', 'human', 'language']
